# Data Collection Notebook

## Installs

In [ ]:
#!pip install socket


## Imports

In [44]:
import socket
import time
import warnings
warnings.filterwarnings("ignore")


## Wifi Setup using socket

In [ ]:
# Arduino board IP address and port
arduino_ip = "192.168.0.152"  # Replace with the actual IP address of your Arduino
arduino_port = 8888


# Create a socket object
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the Arduino
sock.connect((arduino_ip, arduino_port))



## create a list of gestures


In [ ]:
while True:
    user_input_gestures = input('Kindly input a list of gestures seperated by a comma')
    if len(user_input_gestures) < 1:
        print("please provide a list of gestures to continue")
        break
    else: 
        gesture_list = user_input_gestures.split(',')
        gesture_dict = {}
        for gesture_number, gesture in enumerate(gesture_list):
            gesture_dict[gesture.strip().lower()] = gesture_number
        print('kindly check the correct spelling and order of gestures in the dictionary below:')
        print(gesture_dict)
    while True:
        user_input_gesture_confirm = input('''Kindly confirm that the above printed dict is okay: 'yes' or 'no' ''')
        if user_input_gesture_confirm.lower().strip() in ['yes','no']:
            break
        else:
            print('''please input either 'yes' or 'no' ''')
    if user_input_gesture_confirm.lower().strip() == 'yes':
        break

# lets add 'no gesture' to dict
no_gesture_value = len(gesture_dict)
gesture_dict['no gesture'] = no_gesture_value

# setting up data collection procedure for the 

In [78]:
# creating first row (run this cell only once)
raw_sensor_data = open("raw_sensor_data.txt", "w")
column_labels = 'iteration\tgesture_code\tsensor_1_ax\tsensor_1_ay\tsensor_1_az\tsensor_1_gx\tsensor_1_gy\tsensor_1_gz\tsensor_2_ax\tsensor_2_ay\tsensor_2_az\tsensor_2_gx\tsensor_2_gy\tsensor_2_gz\tonboard_ax\tonboard_ay\tonboard_az\tonboard_gx\ttonboard_gy\tonboard_gz\n'
raw_sensor_data.write(column_labels)
raw_sensor_data.close()

(0, 20)


In [74]:
previous_gesture = 0
previous_iteration = 0
raw_sensor_data = open("raw_sensor_data.txt", "w")

In [ ]:
print('lets start collecting data for individual gestures','\nwe will run each gesture at least 100 times to collect sufficient data\n')
for gesture,transmit_val in gesture_dict.items():
    user_start_confirmation = ''
    if transmit_val >= previous_gesture:
        print('please make gesture: ', gesture)
        gesture_iterations = previous_iteration

        while True:
            gesture_iterations +=1
            print('upcoming gesture iteration count: ',gesture_iterations, ' out of 100', 'for gesture: ', gesture)
            user_start_confirmation = input('''enter 'x' to begin data collection for the next iteration, enter 'p' to pause ''')
            gesture_data_collection = []
            if user_start_confirmation == 'x':
                user_save_confirmation = ''
                print('data collection will automatically end after 5 seconds')

                flag_Start_time = time.time()
                while True:
                    if (time.time() - flag_Start_time) > 5:
                        print('iteration complete')
                        break
                    sock.send(str(transmit_val).encode())
                    response = sock.recv(1024)
                    gesture_data_collection.append(response.decode())
                    
                        
                user_save_confirmation = input('would you like to save this data: enter "y" if yes, "p" to pause, leave blank if no')
                if user_save_confirmation == 'y':
                    for a_response in gesture_data_collection:
                        raw_sensor_data_string = str(gesture_iterations) + a_response + '\n'
                        raw_sensor_data_string
                        raw_sensor_data.write(raw_sensor_data_string)
                    print("raw sensor data updated")
                    
                elif user_save_confirmation == 'p':
                    print("previous gesture iteration not save,  pausing data collection")
                    user_start_confirmation = 'p'
                    gesture_iterations -=1
                    break
                else:
                    print("previous gesture iteration not saved, restarting iteration")
                    gesture_iterations -=1
                        

            elif user_start_confirmation == 'p':
                gesture_iterations -=1
                print('paused gesture data collection')
                break

            else:
                gesture_iterations -=1
                print('''please type 'x' to start next iteration of data ''')

        
            if gesture_iterations >100:
                previous_iteration = 0
                break
            
    if user_start_confirmation == 'p':
        previous_gesture = transmit_val
        previous_iteration = gesture_iterations
        print('paused on gesture: ', gesture)
        print('paused on iteration' , previous_iteration)
        break

In [ ]:
#close text file
raw_sensor_data.close()
# Close the socket
sock.close()